# Boston housing price predictor (regression) using DNN

## Step 1: Prepare training and evaluation dataset, create FastEstimator Pipeline

Pipeline can take both data in memory and data in disk. In this example, we are going to use data in memory by loading data with tf.keras.datasets.boston_housing.
The following can be used to get the description of the data:

In [1]:
from sklearn.datasets import load_boston
import pandas as pd

boston = load_boston()
print(boston.DESCR)
print(pd.DataFrame(boston.data).head())

.. _boston_dataset:

Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pu

The same data is also availabe from tensorflow without description and with train and eval data already separated

In [2]:
import tensorflow as tf

(x_train, y_train), (x_eval, y_eval) = tf.keras.datasets.boston_housing.load_data()
print("train shape is {}".format(x_train.shape))
print("train value shape is {}".format(y_train.shape))
print("eval shape is {}".format(x_eval.shape))
print("eval value shape is {}".format(y_eval.shape))

train shape is (404, 13)
train value shape is (404,)
eval shape is (102, 13)
eval value shape is (102,)


Next we need to scale the inputs to the neural network. This is done by using a StandardScaler.

In [3]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_eval = scaler.transform(x_eval)

For in-memory data in Pipeline, the data format should be a nested dictionary like: {"mode1": {"feature1": numpy_array, "feature2": numpy_array, ...}, ...}. Each mode can be either train or eval, in our case, we have both train and eval. feature is the feature name, in our case, we have x and y.

In [4]:
data = {
        "train": {
            "x": x_train, "y": y_train
        },
        "eval": {
            "x": x_eval, "y": y_eval
        }
    }

In [5]:
#Parameters
epochs = 50
batch_size = 32
steps_per_epoch = None

Now we are ready to define Pipeline:

In [6]:
import fastestimator as fe

pipeline = fe.Pipeline(batch_size=batch_size, data=data)

## Step 2: Prepare model, create FastEstimator Network

First, we have to define the network architecture in tf.keras.Model or tf.keras.Sequential. After defining the architecture, users are expected to feed the architecture definition and its associated model name, optimizer and loss name (default to be 'loss') to FEModel.

In [7]:
from tensorflow.keras import layers

def create_dnn():
    model = tf.keras.Sequential()

    model.add(layers.Dense(64, activation="relu", input_shape=(13,)))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(32, activation="relu"))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(16, activation="relu"))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(1, activation="linear"))

    return model

model = fe.build(model_def=create_dnn, model_name="dnn", optimizer="adam", loss_name="loss")

Now we are ready to define the Network: given with a batch data with key x and y, we have to work our way to loss with series of operators. ModelOp is an operator that contains a model.

In [8]:
from fastestimator.op.tensorop import ModelOp, MeanSquaredError

network = fe.Network(
    ops=[ModelOp(inputs="x", model=model, outputs="y_pred"), MeanSquaredError(inputs=("y","y_pred"),outputs="loss")])

## Step 3: Configure training, create Estimator

During the training loop, we want to: 1) measure lowest loss for data data 2) save the model with lowest valdiation loss. Trace class is used for anything related to training loop, we will need to import the ModelSaver trace.

In [9]:
import tempfile
from fastestimator.trace import ModelSaver

model_dir = tempfile.mkdtemp()
traces = [ModelSaver(model_name="dnn", save_dir=model_dir, save_best=True)]

Now we can define the Estimator and specify the training configuation:

In [10]:
estimator = fe.Estimator(network=network, pipeline=pipeline, epochs=50, traces=traces)

## Start Training

In [11]:
estimator.fit()

    ______           __  ______     __  _                 __            
   / ____/___ ______/ /_/ ____/____/ /_(_)___ ___  ____ _/ /_____  _____
  / /_  / __ `/ ___/ __/ __/ / ___/ __/ / __ `__ \/ __ `/ __/ __ \/ ___/
 / __/ / /_/ (__  ) /_/ /___(__  ) /_/ / / / / / / /_/ / /_/ /_/ / /    
/_/    \__,_/____/\__/_____/____/\__/_/_/ /_/ /_/\__,_/\__/\____/_/     
                                                                        


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

FastEstimator-Start: step: 0; total_train_steps: 600; dnn_lr: 0.001; 

## Inferencing

After training, the model is saved to a temporary folder. we can load the model from file and do inferencing on a sample.

In [12]:
import os

model_path = os.path.join(model_dir, 'dnn_best_loss.h5')
trained_model = tf.keras.models.load_model(model_path, compile=False)

Randomly get one sample from validation set and compare the predicted value with model's prediction:

In [13]:
import numpy as np

selected_idx = np.random.randint(0, high=101)
print("test sample idx {}, ground truth: {}".format(selected_idx, y_eval[selected_idx]))

test_sample = np.expand_dims(x_eval[selected_idx], axis=0)

predicted_value = trained_model.predict(test_sample)
print("model predicted value is {}".format(predicted_value))

test sample idx 2, ground truth: 19.0
model predicted value is [[20.183064]]
